In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import json
from numpy import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingClassifier

In [ ]:
ext=('.json')
attackerlist=[]
for files in os.listdir():
    if files.__contains__('A2'):
        attackerlist.append(int(files.split("-")[2]))
print(attackerlist)

In [ ]:
if(os.path.isfile('bsm.csv')):
    os.remove('bsm.csv')
for files in os.listdir():
    if files.endswith(ext):
        atk = pd.read_json(files,lines=True)
        atk.drop(['noise','spd_noise','pos_noise'], inplace=True, axis=1,errors='ignore')
        atk['reciever']=int(files.split("-")[2])
        atk1 = pd.DataFrame(atk['pos'].tolist(), columns=['pos-x1', 'pos-y1', 'pos-z'])
        atk1.drop(['pos-z'], inplace=True, axis=1,errors='ignore')
        atk2 = pd.DataFrame(atk['spd'].tolist(), columns=['spd-x1', 'spd-y1', 'spd-z'])
        atk2.drop(['spd-z'], inplace=True, axis=1,errors='ignore')
        frames1=[atk,atk1,atk2]
        attack=pd.concat(frames1,axis=1)
        attack.drop(['pos','spd'], inplace=True, axis=1,errors='ignore')
        bsm = attack[attack["type"] == 3]
        cs = ['pos-x1','pos-y1','spd-x1','spd-y1']
        g= bsm[cs].shift(-1).add_prefix('prev')
        bsm = pd.concat([bsm, g], axis=1).fillna(0).astype(float)
        bsm['attacker']=0
        bsm.loc[bsm["sender"].isin(attackerlist), "attacker"] = 1
        bsm.drop(['type','rcvTime','sendTime','messageID'], inplace=True, axis=1,errors='ignore')
        bsm.to_csv('bsm.csv', mode='a', header=False, index=False)

In [ ]:
col_names = ['sender','RSSI','reciever','sen-px','sen-py','sen-sx','sen-sy','prev-px','prev-py','prev-sx','prev-sy','attacker']
# load dataset
pi = pd.read_csv("bsm.csv", header=None, names=col_names)

pi['del-px'] = pi['sen-px'] - pi['prev-px']
pi['del-py'] = pi['sen-py'] - pi['prev-py']
pi['del-sx'] = pi['sen-sx'] - pi['prev-sx']
pi['del-sy'] = pi['sen-sy'] - pi['prev-sy']

pi["receiver-px"]= np.random.permutation(pi["sen-px"].values)
pi["receiver-py"]= np.random.permutation(pi["sen-py"].values)
pi["receiver-sx"]= np.random.permutation(pi["sen-sx"].values)
pi["receiver-sy"]= np.random.permutation(pi["sen-sy"].values)

cs = ['receiver-px','receiver-py'	,'receiver-sx'	,'receiver-sy']
g= pi[cs].shift(-1).add_prefix('prev-')
pi = pd.concat([pi, g], axis=1).fillna(0).astype(float)


pi['AoA'] = np.arctan((pi['receiver-py']-pi['sen-py'])/(pi['receiver-px']-pi['sen-px']))
pi['delr-px'] = pi['receiver-px'] - pi['prev-receiver-px']
pi['delr-py'] = pi['receiver-py'] - pi['prev-receiver-py']
pi['delr-sx'] = pi['receiver-sx'] - pi['prev-receiver-sx']
pi['delr-sy'] = pi['receiver-sy'] - pi['prev-receiver-sy']


ctm = pi.pop('attacker');
pi.insert(len(pi.columns),'attacker',ctm)
pi

In [ ]:
X = pi.iloc[:,:-1].values
y = pi.iloc[:, 28].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.99,random_state = 42)

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

knn_y_pred = knn.predict(X_test)

print(confusion_matrix(y_test, knn_y_pred))
print(classification_report(y_test, knn_y_pred))

In [ ]:
rf = RandomForestClassifier(n_estimators=100)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
rf.fit(X_train,y_train)

rf_y_pred = rf.predict(X_test)

print(confusion_matrix(y_test, rf_y_pred))
print(classification_report(y_test, rf_y_pred))

In [ ]:
lr = LogisticRegression()
clfStack = StackingClassifier(estimators=[('knn',knn),('rf',rf)],final_estimator=lr,cv=10)

clfStack.fit(X_train,y_train);

st_pred = clfStack.predict(X_test)

print(confusion_matrix(y_test, st_pred))
print(classification_report(y_test, st_pred))
